In [1]:
import torch
import transformers

print(torch.__version__)
print(transformers.__version__)

2.4.1+cu121
4.44.2


In [9]:
pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.6/375.6 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.6 MB/s eta 0:00:00


In [16]:
pip install google-cloud-bigquery

In [20]:
from google.colab import drive
import os
drive.mount('/content/drive')

os.environ['GOOGLE_APPLICATION_CREDENTIALS']='/content/drive/MyDrive/Colab Notebooks/haystack-436323-fd77f9e327cc.json'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [47]:
import requests
import openai
from transformers import pipeline
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from google.colab import userdata
import time

# Set OpenAI API key for GPT-4
openai.api_key = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
# Timer decorator to track execution time of functions
def timer_decorator(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"Function '{func.__name__}' took {end_time - start_time:.4f} seconds")
        return result
    return wrapper

# 1. Search Patents from Different Data Sources using a Stronger GPT-4 Integration
@timer_decorator
def search_patents_gpt(query, source, api_key=None):
    """
    Searches for patents from different sources based on a flag.
    Uses GPT-4 to process results and enhance the search quality.
    Possible sources: 'lens', 'uspto', 'google_patents', 'wipo'
    """
    print(f"Searching patents with query: {query} on source: {source}")

    if source == "lens":
        patents = search_patents_lens(query, api_key)
    elif source == "uspto":
        patents = search_patents_uspto(query)
    elif source == "google_patents":
        patents = search_google_patents(query)
    elif source == "wipo":
        patents = search_patents_wipo(query, api_key)
    else:
        raise ValueError("Invalid source provided. Choose from: 'lens', 'uspto', 'google_patents', 'wipo'.")

    print(f"Patents retrieved: {patents}")

    if isinstance(patents, list) and len(patents) > 0:
        titles = [patent['title'] for patent in patents]  # Extracting titles from the list of dictionaries
        print(f"Patent titles: {titles}")
        gpt_response = refine_search_with_gpt(query, titles)
        return gpt_response
    else:
        print("No prior art found for comparison.")
        return "No prior art found for comparison."


# 2. Function to Enhance Search Using GPT-4
from openai import OpenAI
client = OpenAI()
@timer_decorator
def refine_search_with_gpt(query, titles):
    """
    Uses GPT-4 to refine and summarize the patent titles retrieved from various sources.
    """
    print(f"Refining search with GPT-4. Query: {query}, Titles: {titles}")

    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a patent expert tasked with finding prior art for a new patent claim."},
                {"role": "user", "content": f"Patent claim: {query}\nHere are some potential prior art titles: {titles}. Which ones are the most relevant, and why?"}
            ]
        )
        gpt_result = response.choices[0].message.content
        #response.choices[0].message['content']
        print(f"GPT-4 response: {gpt_result}")
        return gpt_result
    except Exception as e:
        print(f"Error during GPT-4 API call: {e}")
        return "Error in GPT-4 processing"

# 3. Fetch Patent Data from Lens.org API (requires an API key)
@timer_decorator
def search_patents_lens(query, api_key):
    """
    Searches Lens.org for patents based on a query.
    Requires Lens.org API key for access.
    """
    print(f"Searching Lens.org with query: {query}")

    url = "https://api.lens.org/scholarly/search"
    headers = {
        'Authorization': f'Bearer {api_key}',
        'Content-Type': 'application/json'
    }
    payload = {
        "query": {
            "bool": {
                "must": [
                    {"match": {"text": query}}
                ]
            }
        },
        "size": 10
    }

    response = requests.post(url, json=payload, headers=headers)

    if response.status_code != 200:
        print(f"Error: Received status code {response.status_code}")
        print(f"Response text: {response.text}")
        return {}

    try:
        return response.json()
    except requests.exceptions.JSONDecodeError:
        print("Error: Failed to parse JSON")
        print(f"Response text: {response.text}")
        return {}

# 4. Fetch Patent Data from USPTO (Public API)
@timer_decorator
def search_patents_uspto(query):
    """
    Searches USPTO database for patents using bulk data search.
    """
    print(f"Searching USPTO with query: {query}")

    url = f"https://developer.uspto.gov/ibd-api/v1/patent/application?searchText={query}&rows=10"
    response = requests.get(url)
    return response.json()

# 5. Fetch Patent Data from Google Patents (via BigQuery)
@timer_decorator
def search_google_patents(query):
    """
    Searches Google Patents Public Dataset using BigQuery API.
    Requires Google Cloud setup.
    """
    print(f"Searching Google Patents with query: {query}")

    client = bigquery.Client()

    # Google Patents Public Datasets project and table
    dataset_ref = "patents-public-data.patents.publications"

    # Construct the query
    sql = f"""
        SELECT
            publication_number,
            title_localized,
            ARRAY_TO_STRING(ARRAY_AGG(abstract.text), ' ') AS abstract_text,
            filing_date
        FROM `{dataset_ref}`,
        UNNEST(abstract_localized) AS abstract
        WHERE REGEXP_CONTAINS(abstract.text, r'{query}')
        AND abstract.language = 'en'
        GROUP BY publication_number, title_localized, filing_date
        LIMIT 10
    """

    # Execute the query
    query_job = client.query(sql)

    # Fetch the results
    results = query_job.result()

    # Convert results to a list of dictionaries
    patents = []
    for row in results:
        patents.append({
            "publication_number": row.publication_number,
            "title": row.title_localized,
            "abstract": row.abstract_text,
            "filing_date": row.filing_date
        })

    print(f"Google Patents Results: {patents}")
    return patents

# 6. Fetch Patent Data from WIPO Patentscope API (requires an API key)
@timer_decorator
def search_patents_wipo(query, api_key):
    """
    Searches WIPO Patentscope for patents using the API.
    Requires WIPO API key.
    """
    print(f"Searching WIPO with query: {query}")

    url = f"https://patentscope.wipo.int/search-api/rest/patents?query={query}&rows=10"
    headers = {
        'Authorization': f'Bearer {api_key}'
    }
    response = requests.get(url, headers=headers)
    return response.json()

# 7. LLM for Semantic Similarity (Using Hugging Face Pipeline)
@timer_decorator
def check_novelty(new_patent_claim, prior_art_patents):
    """
    Uses an LLM to check novelty by comparing new patent claims to prior art.
    """
    print(f"Checking novelty for patent claim: {new_patent_claim}")
    print(f"Comparing with prior art patents: {prior_art_patents}")

    # Initialize HuggingFace pipeline for text similarity
    similarity_model = pipeline('feature-extraction', model='sentence-transformers/all-MiniLM-L6-v2')

    # Transform the new patent claim and the prior art patents
    new_claim_embedding = similarity_model(new_patent_claim)[0]
    prior_art_embeddings = [similarity_model(patent)[0] for patent in prior_art_patents]

    # Compare similarity using cosine similarity
    vectorizer = TfidfVectorizer()
    corpus = [new_patent_claim] + prior_art_patents
    tfidf_matrix = vectorizer.fit_transform(corpus)

    # Compute cosine similarities
    cosine_sim = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:])

    print(f"Cosine Similarities: {cosine_sim}")
    return cosine_sim

# 8. Full Pipeline for Patent Novelty Check using GPT-4 for stronger search
@timer_decorator
def patent_novelty_check(patent_claim, api_key, source):
    """
    Pipeline to check the novelty of a patent claim against existing patents.
    Uses GPT-4 to improve the search process.
    """
    print(f"Starting patent novelty check for: {patent_claim}, using source: {source}")

    prior_art = search_patents_gpt(patent_claim, source, api_key)

    if prior_art and prior_art != "No prior art found for comparison.":
        # Assuming that GPT-4 returns a string of relevant patents to compare
        prior_art_list = prior_art.split('\n')
        print(f"Prior art list from GPT-4: {prior_art_list}")
        similarities = check_novelty(patent_claim, prior_art_list)
        print("Cosine Similarities to Prior Art: ", similarities)
    else:
        print("No relevant prior art found for comparison.")
    return prior_art_list, similarities
api_key = "your_lens_org_or_wipo_or_openai_api_key"
patent_claim = "A method for enhancing machine learning model performance by using transfer learning techniques."
patent_claim = 'machine learning'
source = "google_patents"  # Choose from 'lens', 'uspto', 'google_patents', 'wipo'
prior_art_list, similarities = patent_novelty_check(patent_claim, api_key, source)


Starting patent novelty check for: machine learning, using source: google_patents
Searching patents with query: machine learning on source: google_patents
Searching Google Patents with query: machine learning
Google Patents Results: [{'publication_number': 'US-2020320438-A1', 'title': [{'text': 'Prudent ensemble models in machine learning with high precision for use in network security', 'language': 'en', 'truncated': False}], 'abstract': 'Systems and methods include receiving a content item between a user device and a location on the Internet or an enterprise network; utilizing a trained machine learning ensemble model to determine whether the content item is malicious; responsive to the trained machine learning ensemble model determining the content item is malicious or determining the content item is benign but such determining is in a blind spot of the trained ensemble model, performing further processing on the content item; and, responsive to the trained machine learning ensemble

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Cosine Similarities: [[0.30278526 0.         0.25443072 0.         0.54345625 0.
  0.32235069 0.         0.32233819 0.         0.09713204]]
Function 'check_novelty' took 0.5890 seconds
Cosine Similarities to Prior Art:  [[0.30278526 0.         0.25443072 0.         0.54345625 0.
  0.32235069 0.         0.32233819 0.         0.09713204]]
Function 'patent_novelty_check' took 18.8203 seconds


In [64]:
similarities_flat = similarities.flatten()
for i,j in zip(similarities_flat[:5], prior_art_list[:5]):
  print("similarity is: ", round(i,2))
  print(j)

similarity is:  0.3
1. "Prudent ensemble models in machine learning with high precision for use in network security": This patent is directly related to machine learning models, which is the crux of your patent claim.
similarity is:  0.0

similarity is:  0.25
2. "Driving scenario sampling for training/tuning machine learning models for vehicles": This prior art is relevant because it mentions the use of machine learning models which can potentially conflict with your patent claim, depending upon the specifics of your claim.
similarity is:  0.0

similarity is:  0.54
3. "Machine learning device, servomotor control system, and machine learning method": As the title suggests, this prior art talks about a machine learning device and a method related to machine learning. Considering your patent claim is about machine learning, this is a significant piece of prior art.
